In [1]:
# Imports
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from tqdm import tqdm

In [2]:
# Define a request header (to prevent anti-scraping)
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30',
}

In [3]:
url  = "https://arxiv.org/list/cs/pastweek?skip=0&show=100"
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
author_lst   = []
subject_lst  = []
title_lst    = []
abstract_lst = []

In [5]:
for element in soup.find_all('div', {'class': 'list-authors'}):
    author = []
    for sub_element in element.find_all('a'):
        author.append(sub_element.get_text())
    author_lst.append(', '.join(author))
print(f'Length of author_lst is: {len(author_lst)}')
print(f'author_lst[0]: {author_lst[0]}')

Length of author_lst is: 100
author_lst[0]: Tao Zhang, Xiangtai Li, Haobo Yuan, Shunping Ji, Shuicheng Yan


In [6]:
for element in soup.find_all('div', {'class': 'list-subjects'}):
    subject_lst.append(element.get_text()[11:].strip('\n'))
print(f'Length of subject_lst is: {len(subject_lst)}')
print(f'subject_lst[0]: {subject_lst[1]}')

Length of subject_lst is: 100
subject_lst[0]: Computation and Language (cs.CL); Artificial Intelligence (cs.AI); Machine Learning (cs.LG)


In [7]:
for element in soup.find_all('div', {'class': 'list-title mathjax'}):
    title_lst.append(element.get_text()[7:].strip())
print(f'Length of title_lst is: {len(title_lst)}')
print(f'title_lst[-1]: {title_lst[-1]}')

Length of title_lst is: 100
title_lst[-1]: Selective-Stereo: Adaptive Frequency Information Selection for Stereo  Matching


In [8]:
for element in tqdm(soup.find_all('span', {'class': 'list-identifier'})):
    sub_element = element.find('a', {'title': 'Abstract'})
    sub_url = "https://arxiv.org" + sub_element['href']
    sub_page = requests.get(sub_url, headers=headers)
    sub_soup = BeautifulSoup(sub_page.content, 'html.parser')
    sleep(1)
    ab_element = sub_soup.find('blockquote', {'class': 'abstract mathjax'})
    abstract_lst.append(ab_element.get_text()[10:].strip().strip('\n'))
print(f'Length of abstract_lst is: {len(abstract_lst)}')
print(f'abstract_lst[0]: {abstract_lst[0]}')

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [03:24<00:00,  2.05s/it]

Length of abstract_lst is: 100
abstract_lst[0]: In this work, for the first time, we demonstrate that Mamba-based point cloud methods can outperform point-based methods. Mamba exhibits strong global modeling capabilities and linear computational complexity, making it highly attractive for point cloud analysis. To enable more effective processing of 3-D point cloud data by Mamba, we propose a novel Consistent Traverse Serialization to convert point clouds into 1-D point sequences while ensuring that neighboring points in the sequence are also spatially adjacent. Consistent Traverse Serialization yields six variants by permuting the order of x, y, and z coordinates, and the synergistic use of these variants aids Mamba in comprehensively observing point cloud data. Furthermore, to assist Mamba in handling point sequences with different orders more effectively, we introduce point prompts to inform Mamba of the sequence's arrangement rules. Finally, we propose positional encoding based on s

In [9]:
df = pd.DataFrame({'title': title_lst, 'subjects': subject_lst, 'authors': author_lst, 'abstract': abstract_lst})
df.to_csv('./arxiv_cs_top100.csv', index=False)

In [13]:
print(f'Length of all data: {len(df)}')

Length of all data: 100
